# Acknowledgements

이 데이터는 인터넷 사이트를 통해 직접 수집된 데이터입니다.<br>
ML을 이용해 집 가격을 예측하는 모델을 완성해보세요.<br><br>

## 주택 가격 예측 모델 (회귀 모델)<br><br>

# File descriptions

- `train.csv` - 트레이닝 셋<br><br>
- `test.csv` - 테스트 셋<br><br>
- `sampleSubmission.csv` - 케글 점수 체크용 정답 데이터 셋<br><br>

# Data fields

- `ID` - 각 집의 고유한 번호<br><br>
- `ADDRESS` - 집의 주소<br><br>
- `SUBURB` - 동네 이름<br><br>
- `PRICE` - 가격<br><br>
- `BEDROOMS` - 침실의 갯수<br><br>
- `BATHROOMS` - 욕실의 갯수<br><br>
- `GARAGE` - 차고의 수<br><br>
- `LAND_AREA` - 토지 면적<br><br>
- `FLOOR_AREA` - 건물 면적<br><br>
- `BUILD_YEAR` - 건축년도<br><br>
- `CBD_DIST` - Central business district까지의 거리<br><br>
- `NEAREST_STN` - 근처 역 정보<br><br>
- `NEAREST_STN_DIST` - 근처 역까지 거리<br><br>
- `DATE_SOLD` - 판매된 날짜<br><br>
- `POSTCODE` - 우편번호<br><br>
- `LATITUDE` - 위도<br><br>
- `LONGITUDE` - 경도<br><br>
- `NEAREST_SCH` - 근교의 학교<br><br>
- `NEAREST_SCH_DIST` - 근교의 학교까지의 거리<br><br>
- `NEAREST_SCH_RANK` - 근교의 학교까지의 랭킹<br><br>


In [10]:
from matplotlib import rcParams, pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.sparse import hstack
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from warnings import simplefilter
%reload_ext autoreload
%autoreload 2
%matplotlib inline

rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
simplefilter('ignore')

seed = 42
target_col = 'PRICE'

In [11]:
home_df = pd.read_csv('./train.csv')
home_test_df = pd.read_csv('./test.csv')
# 타겟 데이터

In [14]:
home_df

,ID,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,0,9 Crumlin Way,Ridgewood,520000,5,2,4.0,674,266,2007.0,35500,Butler Station,3000,09-2015\r,6030,-31.656206,115.720566,IRENE MCCORMACK CATHOLIC COLLEGE,1.079230,120.0
1,1,37 Sissinghurst Crescent,Landsdale,575000,5,4,2.0,540,301,2014.0,16900,Whitfords Station,8100,07-2018\r,6065,-31.800802,115.867675,LANDSDALE CHRISTIAN SCHOOL,0.667585,NaN
2,2,24 Tees Court,Mindarie,760000,4,2,2.0,781,220,1992.0,33500,Clarkson Station,3500,02-2020\r,6030,-31.682040,115.702600,MINDARIE SENIOR COLLEGE,0.650761,111.0
3,3,31 Brentham Street,Mount Hawthorn,1230000,2,1,2.0,425,185,1938.0,3500,Leederville Station,1500,07-2020\r,6016,-31.926046,115.838181,ARANMORE CATHOLIC COLLEGE,0.484465,106.0
4,4,46/1 Corkhill Street,North Fremantle,2010000,3,2,2.0,546,186,2004.0,12900,North Fremantle Station,800,10-2017\r,6159,-32.032079,115.759626,JOHN CURTIN COLLEGE OF THE ARTS,1.869050,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18505,18505,36 Kentucky Boulevard,Hocking,475000,4,2,2.0,533,198,2007.0,21200,Edgewater Station,4400,04-2018\r,6065,-31.764187,115.824151,WANNEROO SECONDARY COLLEGE,1.104533,131.0
18506,18506,10 Regatta Boulevard,Champion Lakes,485000,3,2,2.0,344,237,2010.0,22100,Kelmscott Station,1900,06-2011\r,6111,-32.114990,115.992834,JOHN WOLLASTON ANGLICAN COMMUNITY SCHOOL,1.816768,40.0
18507,18507,145 Coolgardie Avenue,Redcliffe,642000,3,1,1.0,1016,110,1953.0,8900,Bassendean Station,3100,01-2018\r,6104,-31.931320,115.950060,CYRIL JACKSON SENIOR CAMPUS,2.169696,NaN
18508,18508,12 Mckenzie Road,Samson,491000,4,2,2.0,718,176,1988.0,14600,Murdoch Station,5400,03-2009\r,6163,-32.070861,115.793834,SETON CATHOLIC COLLEGE,0.208669,91.0


In [15]:
home_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18510 entries, 0 to 18509
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                18510 non-null  int64  
 1   ADDRESS           18510 non-null  object 
 2   SUBURB            18510 non-null  object 
 3   PRICE             18510 non-null  int64  
 4   BEDROOMS          18510 non-null  int64  
 5   BATHROOMS         18510 non-null  int64  
 6   GARAGE            17143 non-null  float64
 7   LAND_AREA         18510 non-null  int64  
 8   FLOOR_AREA        18510 non-null  int64  
 9   BUILD_YEAR        16782 non-null  float64
 10  CBD_DIST          18510 non-null  int64  
 11  NEAREST_STN       18510 non-null  object 
 12  NEAREST_STN_DIST  18510 non-null  int64  
 13  DATE_SOLD         18510 non-null  object 
 14  POSTCODE          18510 non-null  int64  
 15  LATITUDE          18510 non-null  float64
 16  LONGITUDE         18510 non-null  float6

In [16]:
home_df.corr()

,ID,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN_DIST,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH_DIST,NEAREST_SCH_RANK
ID,1.000000,0.002149,0.007004,-0.000146,-0.010428,-0.007281,0.006915,0.000651,0.000779,0.003104,0.003222,-0.011983,-0.006936,-0.000778,-0.010885
PRICE,0.002149,1.000000,0.256909,0.386485,0.124276,0.046778,0.539524,-0.148771,-0.357368,-0.099370,-0.162925,0.046722,-0.183514,-0.024030,-0.453937
BEDROOMS,0.007004,0.256909,1.000000,0.558640,0.180447,0.047265,0.536185,0.234275,0.120185,0.105443,0.019795,0.023951,-0.011119,0.093771,0.029626
BATHROOMS,-0.000146,0.386485,0.558640,1.000000,0.181213,0.028357,0.556328,0.345848,0.028758,0.033446,-0.032696,0.048622,-0.094269,0.053522,-0.095940
GARAGE,-0.010428,0.124276,0.180447,0.181213,1.000000,0.042186,0.186341,0.045461,0.042033,0.101591,0.019979,-0.011987,0.085201,0.094102,0.014924
LAND_AREA,-0.007281,0.046778,0.047265,0.028357,0.042186,1.000000,0.066470,0.009614,0.114915,0.201562,0.034252,-0.002033,0.130394,0.245674,0.034244
FLOOR_AREA,0.006915,0.539524,0.536185,0.556328,0.186341,0.066470,1.000000,0.230423,0.022669,0.106088,-0.040491,0.039342,-0.039926,0.117837,-0.136373
BUILD_YEAR,0.000651,-0.148771,0.234275,0.345848,0.045461,0.009614,0.230423,1.000000,0.258332,0.090911,0.038833,0.045702,-0.063877,0.096881,0.151206
CBD_DIST,0.000779,-0.357368,0.120185,0.028758,0.042033,0.114915,0.022669,0.258332,1.000000,0.450933,0.249753,-0.194665,-0.055923,0.379823,0.324407
NEAREST_STN_DIST,0.003104,-0.099370,0.105443,0.033446,0.101591,0.201562,0.106088,0.090911,0.450933,1.000000,0.205185,0.022709,0.497753,0.636990,0.184562


In [17]:
home_df['GARAGE'].fillna(home_df['GARAGE'].value_counts().idxmax(), inplace=True)

home_df

,ID,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,0,9 Crumlin Way,Ridgewood,520000,5,2,4.0,674,266,2007.0,35500,Butler Station,3000,09-2015\r,6030,-31.656206,115.720566,IRENE MCCORMACK CATHOLIC COLLEGE,1.079230,120.0
1,1,37 Sissinghurst Crescent,Landsdale,575000,5,4,2.0,540,301,2014.0,16900,Whitfords Station,8100,07-2018\r,6065,-31.800802,115.867675,LANDSDALE CHRISTIAN SCHOOL,0.667585,NaN
2,2,24 Tees Court,Mindarie,760000,4,2,2.0,781,220,1992.0,33500,Clarkson Station,3500,02-2020\r,6030,-31.682040,115.702600,MINDARIE SENIOR COLLEGE,0.650761,111.0
3,3,31 Brentham Street,Mount Hawthorn,1230000,2,1,2.0,425,185,1938.0,3500,Leederville Station,1500,07-2020\r,6016,-31.926046,115.838181,ARANMORE CATHOLIC COLLEGE,0.484465,106.0
4,4,46/1 Corkhill Street,North Fremantle,2010000,3,2,2.0,546,186,2004.0,12900,North Fremantle Station,800,10-2017\r,6159,-32.032079,115.759626,JOHN CURTIN COLLEGE OF THE ARTS,1.869050,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18505,18505,36 Kentucky Boulevard,Hocking,475000,4,2,2.0,533,198,2007.0,21200,Edgewater Station,4400,04-2018\r,6065,-31.764187,115.824151,WANNEROO SECONDARY COLLEGE,1.104533,131.0
18506,18506,10 Regatta Boulevard,Champion Lakes,485000,3,2,2.0,344,237,2010.0,22100,Kelmscott Station,1900,06-2011\r,6111,-32.114990,115.992834,JOHN WOLLASTON ANGLICAN COMMUNITY SCHOOL,1.816768,40.0
18507,18507,145 Coolgardie Avenue,Redcliffe,642000,3,1,1.0,1016,110,1953.0,8900,Bassendean Station,3100,01-2018\r,6104,-31.931320,115.950060,CYRIL JACKSON SENIOR CAMPUS,2.169696,NaN
18508,18508,12 Mckenzie Road,Samson,491000,4,2,2.0,718,176,1988.0,14600,Murdoch Station,5400,03-2009\r,6163,-32.070861,115.793834,SETON CATHOLIC COLLEGE,0.208669,91.0


In [18]:
home_df['NEAREST_SCH_RANK'].fillna(home_df['NEAREST_SCH_RANK'].mean(), inplace=True)

home_df

,ID,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,0,9 Crumlin Way,Ridgewood,520000,5,2,4.0,674,266,2007.0,35500,Butler Station,3000,09-2015\r,6030,-31.656206,115.720566,IRENE MCCORMACK CATHOLIC COLLEGE,1.079230,120.00000
1,1,37 Sissinghurst Crescent,Landsdale,575000,5,4,2.0,540,301,2014.0,16900,Whitfords Station,8100,07-2018\r,6065,-31.800802,115.867675,LANDSDALE CHRISTIAN SCHOOL,0.667585,72.68705
2,2,24 Tees Court,Mindarie,760000,4,2,2.0,781,220,1992.0,33500,Clarkson Station,3500,02-2020\r,6030,-31.682040,115.702600,MINDARIE SENIOR COLLEGE,0.650761,111.00000
3,3,31 Brentham Street,Mount Hawthorn,1230000,2,1,2.0,425,185,1938.0,3500,Leederville Station,1500,07-2020\r,6016,-31.926046,115.838181,ARANMORE CATHOLIC COLLEGE,0.484465,106.00000
4,4,46/1 Corkhill Street,North Fremantle,2010000,3,2,2.0,546,186,2004.0,12900,North Fremantle Station,800,10-2017\r,6159,-32.032079,115.759626,JOHN CURTIN COLLEGE OF THE ARTS,1.869050,25.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18505,18505,36 Kentucky Boulevard,Hocking,475000,4,2,2.0,533,198,2007.0,21200,Edgewater Station,4400,04-2018\r,6065,-31.764187,115.824151,WANNEROO SECONDARY COLLEGE,1.104533,131.00000
18506,18506,10 Regatta Boulevard,Champion Lakes,485000,3,2,2.0,344,237,2010.0,22100,Kelmscott Station,1900,06-2011\r,6111,-32.114990,115.992834,JOHN WOLLASTON ANGLICAN COMMUNITY SCHOOL,1.816768,40.00000
18507,18507,145 Coolgardie Avenue,Redcliffe,642000,3,1,1.0,1016,110,1953.0,8900,Bassendean Station,3100,01-2018\r,6104,-31.931320,115.950060,CYRIL JACKSON SENIOR CAMPUS,2.169696,72.68705
18508,18508,12 Mckenzie Road,Samson,491000,4,2,2.0,718,176,1988.0,14600,Murdoch Station,5400,03-2009\r,6163,-32.070861,115.793834,SETON CATHOLIC COLLEGE,0.208669,91.00000


In [20]:
home_df['BUILD_YEAR'].fillna(home_df['BUILD_YEAR'].value_counts().idxmax(), inplace=True)

home_df

,ID,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,0,9 Crumlin Way,Ridgewood,520000,5,2,4.0,674,266,2007.0,35500,Butler Station,3000,09-2015\r,6030,-31.656206,115.720566,IRENE MCCORMACK CATHOLIC COLLEGE,1.079230,120.00000
1,1,37 Sissinghurst Crescent,Landsdale,575000,5,4,2.0,540,301,2014.0,16900,Whitfords Station,8100,07-2018\r,6065,-31.800802,115.867675,LANDSDALE CHRISTIAN SCHOOL,0.667585,72.68705
2,2,24 Tees Court,Mindarie,760000,4,2,2.0,781,220,1992.0,33500,Clarkson Station,3500,02-2020\r,6030,-31.682040,115.702600,MINDARIE SENIOR COLLEGE,0.650761,111.00000
3,3,31 Brentham Street,Mount Hawthorn,1230000,2,1,2.0,425,185,1938.0,3500,Leederville Station,1500,07-2020\r,6016,-31.926046,115.838181,ARANMORE CATHOLIC COLLEGE,0.484465,106.00000
4,4,46/1 Corkhill Street,North Fremantle,2010000,3,2,2.0,546,186,2004.0,12900,North Fremantle Station,800,10-2017\r,6159,-32.032079,115.759626,JOHN CURTIN COLLEGE OF THE ARTS,1.869050,25.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18505,18505,36 Kentucky Boulevard,Hocking,475000,4,2,2.0,533,198,2007.0,21200,Edgewater Station,4400,04-2018\r,6065,-31.764187,115.824151,WANNEROO SECONDARY COLLEGE,1.104533,131.00000
18506,18506,10 Regatta Boulevard,Champion Lakes,485000,3,2,2.0,344,237,2010.0,22100,Kelmscott Station,1900,06-2011\r,6111,-32.114990,115.992834,JOHN WOLLASTON ANGLICAN COMMUNITY SCHOOL,1.816768,40.00000
18507,18507,145 Coolgardie Avenue,Redcliffe,642000,3,1,1.0,1016,110,1953.0,8900,Bassendean Station,3100,01-2018\r,6104,-31.931320,115.950060,CYRIL JACKSON SENIOR CAMPUS,2.169696,72.68705
18508,18508,12 Mckenzie Road,Samson,491000,4,2,2.0,718,176,1988.0,14600,Murdoch Station,5400,03-2009\r,6163,-32.070861,115.793834,SETON CATHOLIC COLLEGE,0.208669,91.00000


In [22]:
home_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18510 entries, 0 to 18509
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                18510 non-null  int64  
 1   ADDRESS           18510 non-null  object 
 2   SUBURB            18510 non-null  object 
 3   PRICE             18510 non-null  int64  
 4   BEDROOMS          18510 non-null  int64  
 5   BATHROOMS         18510 non-null  int64  
 6   GARAGE            18510 non-null  float64
 7   LAND_AREA         18510 non-null  int64  
 8   FLOOR_AREA        18510 non-null  int64  
 9   BUILD_YEAR        18510 non-null  float64
 10  CBD_DIST          18510 non-null  int64  
 11  NEAREST_STN       18510 non-null  object 
 12  NEAREST_STN_DIST  18510 non-null  int64  
 13  DATE_SOLD         18510 non-null  object 
 14  POSTCODE          18510 non-null  int64  
 15  LATITUDE          18510 non-null  float64
 16  LONGITUDE         18510 non-null  float6

In [21]:
home_df.isnull().sum()

ID                  0
ADDRESS             0
SUBURB              0
PRICE               0
BEDROOMS            0
BATHROOMS           0
GARAGE              0
LAND_AREA           0
FLOOR_AREA          0
BUILD_YEAR          0
CBD_DIST            0
NEAREST_STN         0
NEAREST_STN_DIST    0
DATE_SOLD           0
POSTCODE            0
LATITUDE            0
LONGITUDE           0
NEAREST_SCH         0
NEAREST_SCH_DIST    0
NEAREST_SCH_RANK    0
dtype: int64

In [23]:
num_cols = [x for x in home_df.columns if home_df[x].dtype in [np.int64, np.float64] and x != target_col]

cat_cols = ['ADDRESS', 'SUBURB', 'NEAREST_STN', 'DATE_SOLD', 'NEAREST_SCH']

print(f'    numeric ({len(num_cols)}):\t{num_cols}')
print(f'categorical ({len(cat_cols)}):\t{cat_cols}')

    numeric (14):	['ID', 'BEDROOMS', 'BATHROOMS', 'GARAGE', 'LAND_AREA', 'FLOOR_AREA', 'BUILD_YEAR', 'CBD_DIST', 'NEAREST_STN_DIST', 'POSTCODE', 'LATITUDE', 'LONGITUDE', 'NEAREST_SCH_DIST', 'NEAREST_SCH_RANK']
categorical (5):	['ADDRESS', 'SUBURB', 'NEAREST_STN', 'DATE_SOLD', 'NEAREST_SCH']


In [24]:
print(cat_cols)

['ADDRESS', 'SUBURB', 'NEAREST_STN', 'DATE_SOLD', 'NEAREST_SCH']


In [25]:
pd.DataFrame(home_df['ADDRESS'].value_counts())

,ADDRESS
28 Bent Street,2
39 East Street,2
8 Knutsford Street,2
2 Morris Street,2
11 Second Avenue,2
...,...
42 Grove Street,1
88A Redwood Crescent,1
9 Eriskay Terrace,1
6 Gould Place,1


In [26]:
pd.DataFrame(home_df['SUBURB'].value_counts())

,SUBURB
Bertram,138
Bennett Springs,127
Butler,125
Merriwa,124
Iluka,116
...,...
O'Connor,3
Treeby,2
Munster,1
Welshpool,1


In [27]:
pd.DataFrame(home_df['NEAREST_STN'].value_counts())

,NEAREST_STN
Midland Station,2252
Warwick Station,946
Cockburn Central Station,906
Armadale Station,769
Butler Station,670
...,...
Belmont Park Station,20
Mosman Park Station,18
McIver Station,12
City West Station,9


In [28]:
pd.DataFrame(home_df['DATE_SOLD'].value_counts())

,DATE_SOLD
10-2020\r,329
09-2020\r,329
07-2019\r,318
11-2020\r,316
05-2018\r,300
...,...
04-2005\r,1
09-1988\r,1
09-1989\r,1
11-1992\r,1


In [29]:
pd.DataFrame(home_df['NEAREST_SCH'].value_counts())

,NEAREST_SCH
SWAN VIEW SENIOR HIGH SCHOOL,499
KIARA COLLEGE,417
JOSEPH BANKS SECONDARY COLLEGE,386
ATWELL COLLEGE,380
SWAN VALLEY ANGLICAN COMMUNITY SCHOOL,310
...,...
ELLENBROOK CHRISTIAN COLLEGE,10
ST GEORGE'S ANGLICAN GRAMMAR SCHOOL,8
FAIRBRIDGE COLLEGE,7
METHODIST LADIES' COLLEGE,3


In [30]:
features = num_cols + cat_cols
print(features)

['ID', 'BEDROOMS', 'BATHROOMS', 'GARAGE', 'LAND_AREA', 'FLOOR_AREA', 'BUILD_YEAR', 'CBD_DIST', 'NEAREST_STN_DIST', 'POSTCODE', 'LATITUDE', 'LONGITUDE', 'NEAREST_SCH_DIST', 'NEAREST_SCH_RANK', 'ADDRESS', 'SUBURB', 'NEAREST_STN', 'DATE_SOLD', 'NEAREST_SCH']


In [36]:
from sklearn.feature_extraction import FeatureHasher

fh = FeatureHasher(n_features=1024, input_type='string')

X = hstack([home_df[num_cols]] + [fh.fit_transform(home_df[col]) for col in cat_cols])

print(X.shape)

(18510, 5134)
